In [1]:
import pandas as pd

## 예제 8-4  사용자 정의 함수를 apply 메소드로  처리하기

In [2]:
pd.Series.__dict__['apply']

<function pandas.core.series.Series.apply>

In [3]:
pd.DataFrame.__dict__['apply']

<function pandas.core.frame.DataFrame.apply>

In [4]:
import numpy as np

In [36]:
df = pd.DataFrame ({'a' : np.random.randn(6),
                 'b' : ['철수', '영희'] * 3,
                 'c' : np.random.randn(6)})

In [37]:
df

,a,b,c
0,-0.897123,철수,0.364861
1,0.361416,영희,1.942530
2,-0.339400,철수,0.143598
3,-2.247471,영희,0.906999
4,-0.430916,철수,-1.326798
5,-1.058397,영희,-0.711945


In [38]:
def my_test(a, c):
    return a % c

In [39]:
df['Value'] = df.apply(lambda row: my_test(row['a'], row['c']), axis=1)

In [40]:
df

,a,b,c,Value
0,-0.897123,철수,0.364861,0.197460
1,0.361416,영희,1.942530,0.361416
2,-0.339400,철수,0.143598,0.091393
3,-2.247471,영희,0.906999,0.473525
4,-0.430916,철수,-1.326798,-0.430916
5,-1.058397,영희,-0.711945,-0.346452


In [41]:
df.a % df.c

0    0.197460
1    0.361416
2    0.091393
3    0.473525
4   -0.430916
5   -0.346452
dtype: float64

In [42]:
def my_test2(row):
    return row['a'] % row['c']

In [43]:
df['Value 2'] = df.apply(my_test2, axis=1)

In [44]:
df

,a,b,c,Value,Value 2
0,-0.897123,철수,0.364861,0.197460,0.197460
1,0.361416,영희,1.942530,0.361416,0.361416
2,-0.339400,철수,0.143598,0.091393,0.091393
3,-2.247471,영희,0.906999,0.473525,0.473525
4,-0.430916,철수,-1.326798,-0.430916,-0.430916
5,-1.058397,영희,-0.711945,-0.346452,-0.346452


In [113]:
sample = pd.DataFrame({'임의의값':[10,100,40] })

In [114]:
sample

,임의의값
0,10
1,100
2,40


In [115]:
sample = pd.concat([sample]*100000).reset_index(drop=True)

In [116]:
sample.head()

,임의의값
0,10
1,100
2,40
3,10
4,100


In [117]:
sample.shape

(300000, 1)

In [119]:
%timeit sample['임의의값'].apply(lambda x: np.nan if x < 90 else x)

168 ms ± 45.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [121]:
%timeit sample['임의의값'].mask(sample['임의의값'] < 90, np.nan)

9.9 ms ± 2.99 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [122]:
sample1 = sample.copy()

In [123]:
sample1 = sample1['임의의값'].apply(lambda x: np.nan if x < 90 else x)

In [127]:
sample1.head()

0      NaN
1    100.0
2      NaN
3      NaN
4    100.0
Name: 임의의값, dtype: float64

In [124]:
sample2 = sample.copy()

In [125]:
sample2 = sample2['임의의값'].mask(sample['임의의값'] < 90, np.nan)

In [126]:
sample2.head()

0      NaN
1    100.0
2      NaN
3      NaN
4    100.0
Name: 임의의값, dtype: float64

In [128]:
(sample1 == sample2).shape

(300000,)

In [129]:
sample1.isnull().sum(), sample1.notnull().sum()

(200000, 100000)

In [130]:
(sample1 == sample2).sum()

100000

## 예제 8-5  사용자 정의 함수를 map,applymap메소드로  처리하기

In [70]:
pd.Series.__dict__['map']

<function pandas.core.series.Series.map>

In [71]:
try : 
    pd.Series.__dict__['applymap']
except KeyError as e :
    print("KeyError : ",e)

KeyError :  'applymap'


In [72]:
try : 
    pd.DataFrame.__dict__['map']
except KeyError as e :
    print("KeyError : ",e)

KeyError :  'map'


In [73]:
pd.DataFrame.__dict__['applymap']

<function pandas.core.frame.DataFrame.applymap>

In [74]:
import numpy as np

In [75]:
ser = pd.Series(np.random.randn(6))

In [76]:
ser

0    0.382457
1    0.122235
2    0.320853
3    0.761753
4   -0.338355
5   -0.858526
dtype: float64

In [77]:
def map_test(a):
    print(" 원소별 처리")
    return np.abs(a)

In [78]:
ser.map(map_test)

 원소별 처리
 원소별 처리
 원소별 처리
 원소별 처리
 원소별 처리
 원소별 처리


0    0.382457
1    0.122235
2    0.320853
3    0.761753
4    0.338355
5    0.858526
dtype: float64

## 예제 8-6 pipe 메소드 처리하기

In [3]:
import numpy as np

In [7]:
df = pd.DataFrame({"name": ['김상갑', '임종문', '조현웅'],
                   "program language": [np.nan, 'Python', 'Scala'],
                   "born": [pd.NaT, pd.Timestamp("1966-04-25"),
                             pd.NaT]})

In [8]:
def name_length(df) :
    df['length'] = df.name.str.len()
    return df

In [9]:
df.pipe(name_length)

,born,name,program language,length
0,NaT,김상갑,NaN,3
1,1966-04-25,임종문,Python,3
2,NaT,조현웅,Scala,3


In [10]:
def born_fillna(df) :
    df['born'] = df['born'].fillna(pd.Timestamp("1967-04-25"))
    return df

In [11]:
df.pipe(born_fillna)

,born,name,program language,length
0,1967-04-25,김상갑,NaN,3
1,1966-04-25,임종문,Python,3
2,1967-04-25,조현웅,Scala,3


In [13]:
def pl_fillna(df) :
    df['program language'] = df['program language'].fillna("Java")
    return df

In [17]:
df.pipe(pl_fillna)

,born,name,program language,length
0,1967-04-25,김상갑,Java,3
1,1966-04-25,임종문,Python,3
2,1967-04-25,조현웅,Scala,3


In [15]:
df1 = pd.DataFrame({"name": ['김상갑', '임종문', '조현웅'],
                   "program language": [np.nan, 'Python', 'Scala'],
                   "born": [pd.NaT, pd.Timestamp("1966-04-25"),
                             pd.NaT]})

In [18]:
df1.pipe(name_length).pipe(born_fillna).pipe(pl_fillna)

,born,name,program language,length
0,1967-04-25,김상갑,Java,3
1,1966-04-25,임종문,Python,3
2,1967-04-25,조현웅,Scala,3


In [64]:
df2 = pd.DataFrame()

In [65]:
df2['name'] = ['은옥찬', '은석찬', '은옥주']
df2['gender'] = ['Male', 'Male', 'Female']
df2['age'] = [31, 32, 19]

In [66]:
df2

,name,gender,age
0,은옥찬,Male,31
1,은석찬,Male,32
2,은옥주,Female,19


In [67]:
def mean_age_by_group(dataframe, col):
    return dataframe.groupby(col).mean()

In [68]:
mean_age_by_group(df2, 'gender')

,age
gender,
Female,19.0
Male,31.5


In [69]:
def uppercase_column_name(dataframe):
    dataframe.columns = dataframe.columns.str.upper()
    return dataframe

In [70]:
uppercase_column_name(df2)

,NAME,GENDER,AGE
0,은옥찬,Male,31
1,은석찬,Male,32
2,은옥주,Female,19


In [61]:
df3 = pd.DataFrame()

In [62]:
df3['name'] = ['구옥찬', '구석찬', '구옥주']
df3['gender'] = ['Male', 'Male', 'Female']
df3['age'] = [31, 32, 19]

In [63]:
(df3.pipe(mean_age_by_group, col='gender')
   .pipe(uppercase_column_name))

,AGE
gender,
Female,19.0
Male,31.5
